**Python project by Ahalian R., Spiridonov D., Starikovich I.**

In [2]:
pip install --upgrade nbformat

Note: you may need to restart the kernel to use updated packages.


In [3]:
# библиотеки для парсинга:
import requests 
from bs4 import BeautifulSoup
# библиотеки для анализа данных:
import numpy as np
import pandas as pd
# библиотека для визуализации данных:
import nbformat
import plotly.express as px

In [4]:
# подключаемся к сайту с программами ДПО НИУ ВШЭ, странице №1
url = 'https://www.hse.ru/edu/dpo/?page=1'
resp = requests.get(url)
cont = resp.content
soup = BeautifulSoup(cont)

In [5]:
# получаем ссылку на последнюю страницу с программамми ДПО
# выгружаем количество страниц (pnum)
last = soup.find('button', attrs={'class': "dpob-button_gray dpob-nav__button_last dpob-button dpob-nav__button"})
pnum = int(last.text)

In [6]:
# создаём список hseurls и добавляем в него ссылки на все страницы с программами ДПО
hseurls = []
for i in range(1, pnum + 1):
    hseurls.append(f'https://www.hse.ru/edu/dpo/?page={i}')   
print(hseurls)

['https://www.hse.ru/edu/dpo/?page=1', 'https://www.hse.ru/edu/dpo/?page=2', 'https://www.hse.ru/edu/dpo/?page=3', 'https://www.hse.ru/edu/dpo/?page=4', 'https://www.hse.ru/edu/dpo/?page=5', 'https://www.hse.ru/edu/dpo/?page=6', 'https://www.hse.ru/edu/dpo/?page=7', 'https://www.hse.ru/edu/dpo/?page=8', 'https://www.hse.ru/edu/dpo/?page=9', 'https://www.hse.ru/edu/dpo/?page=10', 'https://www.hse.ru/edu/dpo/?page=11', 'https://www.hse.ru/edu/dpo/?page=12', 'https://www.hse.ru/edu/dpo/?page=13', 'https://www.hse.ru/edu/dpo/?page=14', 'https://www.hse.ru/edu/dpo/?page=15', 'https://www.hse.ru/edu/dpo/?page=16', 'https://www.hse.ru/edu/dpo/?page=17', 'https://www.hse.ru/edu/dpo/?page=18', 'https://www.hse.ru/edu/dpo/?page=19', 'https://www.hse.ru/edu/dpo/?page=20', 'https://www.hse.ru/edu/dpo/?page=21', 'https://www.hse.ru/edu/dpo/?page=22', 'https://www.hse.ru/edu/dpo/?page=23', 'https://www.hse.ru/edu/dpo/?page=24']


In [7]:
# Попробуем найти название первой программы со страницы №1
soup.find('span', attrs={'class': "dpob-card__title-inner"}).text

'Проектирование бренд-медиа для специалистов по PR, маркетингу и корпоративным коммуникациям'

In [8]:
# Попробуем найти длительность первой программы со страницы №1
soup.find('li', attrs={'class': "dpob-icon_duration dpob-icon dpob-card__icon", 'title': "Продолжительность"}).text

'4 недели'

In [9]:
# функция разбивает спаршенную строку с сайта содержащую длительность курса
# длительность на сайте указывается в днях или неделях или месяцах
# данная функция определяет в каких единицах измерена длительность и
# преобразует её в количество дней (если длительность указана в месяцах,
# то количество дней в месяце принимается равным за 30)
def count_days(str):
    ans = 0
    if str.split()[1] in ('дней', 'дня', 'день'):
        ans = float(str.split()[0])
    elif str.split()[1] in ('недель', 'неделя', 'недели'):
        ans = float(str.split()[0])*7
    elif str.split()[1] in ('месяц', 'месяцев', 'месяца'):
        ans = float(str.split()[0])*30
    return ans

In [10]:
# функция получает на вход адрес сайта, точнее конкретной страницы сайта ДПО
# обрабатывает используя BeautifulSoup и на выход выдаёт название программы
def get_courses_titles(url):
    ans = []
    page = requests.get(url)
    soup = BeautifulSoup(page.content)
    for el in soup.find_all('span', attrs={'class': "dpob-card__title-inner"}):
        ans.append(el.text)
    return ans

In [11]:
# функция получает на вход адрес сайта, точнее конкретной страницы сайта ДПО
# обрабатывает используя BeautifulSoup и на выход выдаёт длительность программы в днях
# для рассчёта количества дней используется функция count_days
# конструкция if else используется для сохранения программ, у которых не указана
# длительность, значение None
def get_courses_duration(url):
    ans = []
    page = requests.get(url)
    soup = BeautifulSoup(page.content)
    for el in soup.find_all('div', attrs={'class': "dpob-card__bottom"}):
        codepart = el.find('li', attrs={'class': "dpob-icon_duration dpob-icon dpob-card__icon", 'title': "Продолжительность"})
        if codepart is not None:
            ans.append(count_days(codepart.text))
        else:
            ans.append(None)
    return ans

In [12]:
# цикл for перебирает все страницы сайта с ДПО
# из каждого сайта парсятся названия программ в список titles
# и длительности соответствующих программ в список durations
titles = []
durations = []
for url in hseurls:
    titles.extend(get_courses_titles(url))
    durations.extend(get_courses_duration(url))

In [13]:
# названия и длительность объединяются в coursesdf - датафрейм библиотеки Pandas
coursesdf = pd.DataFrame({'title': titles, 'days': durations})

In [43]:
coursesdf.shape[0] - coursesdf.dropna().shape[0]

0

In [38]:
coursesdf.dropna(inplace=True)

Мы удалили строки с отсутствующими значениями, их всего 5.

In [39]:
# первые 5 строк полученного датафрейма
# title = название, days = длительность в днях
coursesdf.head()

,title,days
0,Проектирование бренд-медиа для специалистов по...,28.0
1,Финансовое моделирование,6.0
2,Английский язык (уровни Pre-upper-intermediate...,60.0
3,"Контрактная система в сфере закупок товаров, р...",21.0
4,Теория и практика разработки инструментов оцен...,10.0


In [40]:
# всего строк в датафрейме coursesdf
coursesdf.shape[0]

460

In [16]:
# выведем описателньые статистики
coursesdf.describe()

,days
count,460.000000
mean,88.886957
std,126.808659
min,1.000000
25%,8.000000
50%,42.000000
75%,91.000000
max,900.000000


Выведем 5 наиболее встречающихся значений длительности курса и сколько раз они встречались. 

In [48]:
coursesdf.value_counts('days').head()

days
8.0     45
84.0    32
60.0    28
21.0    20
6.0     18
dtype: int64

In [17]:
px.histogram(coursesdf, x='days', nbins=30)

In [18]:
px.box(coursesdf, y='days')

Есть несколько нетипичных наблюдений, при чём только характерно большие значения, что ожидаемо, поскольку количество дней не может быть меньше 1 в данном случае.  
В целом сведения совпадают с тем, что мы видели выше:  
Медиана = квартиль 0.5 = 42  
Верхний квартиль = квантиль 0.75 = 91
Нижний квартиль = квантиль 0.25 = 8

Удалим выбросы используя межквартильный размах и создадим датафрейм без этих выбросов = *coursesdfout*

In [35]:
q1= coursesdf['days'].quantile(0.25)
q3 = coursesdf['days'].quantile(0.75)
IQR = q3 - q1
upper = q3 + 1.5 * IQR
lower = q1 - 1.5 * IQR
coursesdfout = coursesdf[(coursesdf['days'] > lower) & (coursesdf['days'] < upper)].sort_values(by='days')
coursesdfout.shape[0]

398

In [37]:
coursesdf.shape[0] - coursesdfout.shape[0]

67

Было удалено 67 записей

In [36]:
coursesdfout.describe()

,days
count,398.000000
mean,47.678392
std,49.076172
min,1.000000
25%,8.000000
50%,28.000000
75%,84.000000
max,210.000000


In [31]:
px.histogram(coursesdfout, x='days', nbins=30)

In [30]:
px.box(coursesdfout, y='days')

После удаления выбросов мы можем посмотреть на распределение курсов дополнительного образования в ином ключе.